# Extract Metadata from Geo Databases, CSVs and Images
This Notebook contains all the codes to extract metadata for 
- Geodatabases
- Shapefiles
- CSV files
- Images

This notebook is best for step by step understanding of the workflow.
* To run entire workflow, use `extract_all_metadata.py` instead.

Contributor: Selorm Komla Darkey (skdarkey@gmail.com)

In [38]:
import geopandas as gpd
import pandas as pd
import fiona 
from fiona.errors import DriverError
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import shapely
from shapely.geometry import shape
from shapely.errors import GeometryTypeError


In [39]:
# Global Vars
ROOT_DIRS = [r'D:\NEOM_PROJECT']
OUTPUT_GDB_METADATA_CSV = r'C:\PERSONAL\UK PHD\NEOM_PROJECT\gdb_layer_metadata.csv'
OUTPUT_SHP_METADATA_CSV = r"C:\PERSONAL\UK PHD\NEOM_PROJECT\shp_layer_metadata.csv"
OUTPUT_IMGS_METADATA_CSV = r"C:\PERSONAL\UK PHD\NEOM_PROJECT\images_layer_metadata.csv"
OUTPUT_CSV_METADATA_CSV = r"C:\PERSONAL\UK PHD\NEOM_PROJECT\csv_xlsx_tables_metadata.csv"

# Lists to be used in organising the metadata
SPECIES_TYPES = ["Corals", "Dugong", "Turtles", "Flying Fish", "Flora And Fauna", "Bird", "Cetaceans"]
ACTIVITY_TYPES = ["Restoration", "Survey", "Study", "Species Management", "Species_Recovery"]
DATE_COLUMNS = ["Timestamp", "Date_", "StartDate", "EndDate"]

CRS = '32636' # for Neom Area


In [40]:
# function to return geodatabase / files paths in list 
def get_geodbs_to_list(root_dirs, files_endwith='gdb'):
    """This function walks through directories and grabs all geodatabases"""
    db_names = []
    db_paths = []

    for root_dir in root_dirs:
        for dirpath, dirnames, filenames in os.walk(root_dir):
            for f in dirnames:
                if f.endswith(f"{files_endwith}"):
                    file_path = os.path.join(dirpath, f)
                    
                    # Extract parts 
                    parts = os.path.normpath(file_path).split(os.sep)  
                    name = parts[-2]   

                    db_names.append(name)
                    db_paths.append(file_path)

    return db_names, db_paths


In [41]:
# Get geo dbs to list
gdb_names, gdb_paths = get_geodbs_to_list(root_dirs=ROOT_DIRS)

print(gdb_names)
print(gdb_paths)

['Gulf of Aqaba (Magna)', 'Gulf of Aqaba (Magna)', '83393_20251006_Submission', '83393_20251008_Submission', 'Spatial Data', 'Spatial Data', 'Spatial Data', 'Spatial Data', 'Spatial Data', 'Spatial Data', 'Spatial Data', 'Spatial Data', 'Spatial Data', 'Spatial Data', 'Spatial Data', 'Spatial Data', 'Spatial Data', 'Spatial Data', 'Spatial Data', 'Spatial Data', 'Spatial Data', 'Spatial Data', 'Spatial Data', '02_83242_Marine_Species_Recovery_Conservation', '06_83242_MSR_CAP_Dugong', '16_83291_Marine_Megafauna_Offshore_Bird_Survey', '27_83242_MSR_CAP_Dugong_IR_Rev01', '28_83291_NEOM_Marine_Megafauna', '29_83374_NEOM_Marine_Megafauna_Aerial_Survey', '30_83437-NEOM_FDSA_Megafauna', '31_83474_Dugong_M_Megafauna_DVL_Survey', '32_83214_Islands_Turtle_Surveys_Tagging', '04_NEOM_FinalData_GDB_20211209', 'Complete dataset for birds and megafauna', '2025-05-11_All Marine Megafauna & Avifauna Geodatabase KMZ', 'FINAL', 'KBD.gdb', 'Avifauna_Artificial_Nest_IR_Rev01', '83474_20240823_Submission', 

In [42]:
# how many geo databases do we have
len(gdb_paths)

48

In [43]:
# find layers in gdb
def get_gdb_layers(gdb_paths):
    """
    Returns a dictionary mapping each geodatabase path
    to a list of its layer names.
    """
    layers_dict = {}
    rows = []

    for gdb_path in gdb_paths:
        try:
            layers = fiona.listlayers(gdb_path)
            layers_dict[gdb_path] = list(layers)

        except Exception as e:
            print(f"Skipping {gdb_path}: {e}")
            layers_dict[gdb_path] = []  # keep record of failed GDBs
            continue

    # return layers_dict
    for gdb_path, layers in layers_dict.items():
        if layers:
            for layer in layers:
                rows.append({
                    "geodatabase": gdb_path,
                    "layer": layer
                })
        else:
            rows.append({
                "geodatabase": gdb_path,
                "layer": None
            })

        # save a csv
    # df = pd.DataFrame(rows)
    # df.to_csv(f"{output_layers_csv_path}", index=False)
    
    # return layers
    return rows

# function to find matching species and activity types
def find_match(values, parts):
    """This function finds matches 
        between two sets of strings. It is used 
        here to find species and activity types from 
        parts of the file name.
    I """
    for v in values:
        v_norm = v.lower().replace(" ", "_")
        if any(v_norm in p.replace(" ", "_") for p in parts):
            return v
    return None


In [44]:
# get each geo db and its containing layers
layers = get_gdb_layers(gdb_paths)
layers

Skipping D:\NEOM_PROJECT\Marine_New_Data\Complete dataset for birds and megafauna\KBD.gdb: Failed to open dataset (flags=68): D:\NEOM_PROJECT\Marine_New_Data\Complete dataset for birds and megafauna\KBD.gdb


[{'geodatabase': 'D:\\NEOM_PROJECT\\Flying Fish Study\\Gulf of Aqaba (Magna)\\FFT_GIS_NEOM1000_HabitatPatchLayerMagna_V1_20250417.gdb',
  'layer': 'enSuperFine_HabitatsMagna_P'},
 {'geodatabase': 'D:\\NEOM_PROJECT\\Flying Fish Study\\Gulf of Aqaba (Magna)\\FFT_GIS_NEOM1000_ImageTransectDataMagna_V2_20250508.gdb',
  'layer': 'enImageLevel_DataMagnaP'},
 {'geodatabase': 'D:\\NEOM_PROJECT\\Flying Fish Study\\Gulf of Aqaba (Magna)\\FFT_GIS_NEOM1000_ImageTransectDataMagna_V2_20250508.gdb',
  'layer': 'enIncidental_SightingsMagnaP'},
 {'geodatabase': 'D:\\NEOM_PROJECT\\Flying Fish Study\\Gulf of Aqaba (Magna)\\FFT_GIS_NEOM1000_ImageTransectDataMagna_V2_20250508.gdb',
  'layer': 'enTransect_LocationsMagnaP'},
 {'geodatabase': 'D:\\NEOM_PROJECT\\Flying Fish Study\\Gulf of Aqaba (Magna)\\FFT_GIS_NEOM1000_ImageTransectDataMagna_V2_20250508.gdb',
  'layer': 'enTransectLevel_DataMagnaL'},
 {'geodatabase': 'D:\\NEOM_PROJECT\\KBD Coral Restoration Data\\83393_20251006_Submission\\NEOM_TREP_Coral_Res

In [45]:
# read the output 
lyrs_df = pd.DataFrame(layers)

lyrs_df.head()

,geodatabase,layer
0,D:\NEOM_PROJECT\Flying Fish Study\Gulf of Aqab...,enSuperFine_HabitatsMagna_P
1,D:\NEOM_PROJECT\Flying Fish Study\Gulf of Aqab...,enImageLevel_DataMagnaP
2,D:\NEOM_PROJECT\Flying Fish Study\Gulf of Aqab...,enIncidental_SightingsMagnaP
3,D:\NEOM_PROJECT\Flying Fish Study\Gulf of Aqab...,enTransect_LocationsMagnaP
4,D:\NEOM_PROJECT\Flying Fish Study\Gulf of Aqab...,enTransectLevel_DataMagnaL


### READ EACH LAYER AND DERIVE ITS META DATA
- crs
- spatial extent / bbox
- geometry types
- field names
- field data types
- geometry types


Read and Check one layer to see the records

In [46]:
# read one database layer to check
# gdf = gpd.read_file(lyrs_df.geodatabase[0], layer=lyrs_df.layer[0])
# gdf.head()

In [47]:
# What's the length of this dataset
# gdf.shape

Extract meta data for all layers

In [50]:
# function to extract layers meta data

def extract_gdb_layer_metadata(
    layers_df,
    output_csv
):
    """
    Reads geodatabase layers and extracts metadata safely.
    
    Parameters
    ----------
    layers_df : pd.DataFrame
        Must contain columns: ['geodatabase', 'layer']
    output_csv : str
        Path to save metadata CSV

    
    Returns
    -------
    pd.DataFrame
        Detailed metadata table
    """

    records = []

    for idx, row in layers_df.iterrows():
        gdb = row["geodatabase"]
        layer = row["layer"]

        meta = {
            "geodatabase": gdb,
            "layer": layer,
            "status": "success",
            "error": None
        }
   

        try:
            # ---- Read layer ----
            gdf = gpd.read_file(gdb, layer=layer)

            # # update to oriental bbox
            if gdf.crs is None:
                raise ValueError("Layer has no CRS defined")

            # Reproject if geographic (degrees)
            epsg = gdf.crs.to_epsg()
            if epsg == 4326:
                gdf = gdf.to_crs(32636)  # choose correct UTM zone

            # Clean invalid geometries 
            gdf["geometry"] = gdf.geometry.make_valid()

            # Dissolve all features
            geom = gdf.geometry.union_all()

            # Oriented bounding box
            obb = geom.minimum_rotated_rectangle
            obb_coords = list(obb.exterior.coords)[:4]

            # ---- Spatial metadata ----
            meta["crs"] = str(gdf.crs)
            meta["epsg"] = gdf.crs.to_epsg() if gdf.crs else None

            meta["geometry_types"] = ", ".join(sorted(gdf.geom_type.unique()))
            meta["bbox"] = list(gdf.total_bounds)
            meta["obb_bbox"] = obb_coords
            meta["geometry"] = gdf.geometry  # adding geometry 


            # ---- Feature-level metadata ----
            meta["feature_count"] = len(gdf)
            meta["has_geometry"] = "geometry" in gdf.columns

            # ----- Temporal meta data -----
            existing_date_cols = [col for col in DATE_COLUMNS if col in gdf.columns]

            meta["has_timestamp"] = len(existing_date_cols) > 0

            if meta["has_timestamp"]:
                for col in existing_date_cols:
                    gdf[col] = pd.to_datetime(gdf[col], errors="coerce")

                all_dates = pd.concat([gdf[col].dropna() for col in existing_date_cols])

                if not all_dates.empty:
                    meta["min_date"] = all_dates.min()
                    meta["max_date"] = all_dates.max()
                else:
                    meta["min_date"] = None
                    meta["max_date"] = None


            # ---- Attribute metadata ----
            meta["field_count"] = len(gdf.columns)
            meta["field_names"] = ", ".join(gdf.columns)

            meta["field_types"] = ", ".join(
                f"{col}:{dtype}"
                for col, dtype in gdf.dtypes.items()
            )
            # ---- Filtering metadata ----
            parts = layer.split('_')
            meta["first_word"] = f"{parts[0]}_{parts[1]}"

            gdb_parts = gdb.split(os.sep)
           
            # Normalize path parts once
            gdb_parts_lower = [p.lower() for p in gdb_parts]

            # ----- Species detection -----
            meta["Species"] = find_match(SPECIES_TYPES, gdb_parts_lower)
            meta["activity"] = find_match(ACTIVITY_TYPES, gdb_parts_lower)

            # ---- Derived metadata ----
            meta["memory_mb"] = round(
                gdf.memory_usage(deep=True).sum() / (1024 ** 2), 3
            )

            # ---- Z / M detection (best-effort) ----
            try:
                meta["has_z"] = gdf.geometry.has_z.any()
            except Exception:
                meta["has_z"] = None

        except (DriverError, PermissionError) as e:
            meta["status"] = "skipped"
            meta["error"] = str(e)

        except Exception as e:
            meta["status"] = "failed"
            meta["error"] = str(e)

        records.append(meta)

    # ---- Create DataFrame ----
    meta_df = pd.DataFrame(records)

    # ---- Save CSV ----
    meta_df.to_csv(output_csv, index=False, encoding="utf-8")

    # return meta_df


In [51]:
# extract meta data for each geo database layer and save csv
extract_gdb_layer_metadata(
    lyrs_df,
    output_csv=OUTPUT_GDB_METADATA_CSV
)

c:\PERSONAL\UK PHD\NEOM_PROJECT\neom\Lib\site-packages\pyogrio\raw.py:200: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiLineString' is converted to 'MultiLineString Z'
  return ogr_read(
C:\Users\sd923\AppData\Local\Temp\ipykernel_91352\1421235877.py:109: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_dates = pd.concat([gdf[col].dropna() for col in existing_date_cols])
C:\Users\sd923\AppData\Local\Temp\ipykernel_91352\1421235877.py:109: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_dates = pd.concat([

## Get all images meta data table

In [25]:
# function to grab all images in the directory 
def get_images_to_list(root_dirs, files_endwith):
    """This function walks through directories and grabs all image files"""
    db_names = []
    db_paths = []

    for root_dir in root_dirs:
        for dirpath, dirnames, filenames in os.walk(root_dir):
            for f in filenames:
                for i in files_endwith:
                    if f.endswith(f"{i}"):   #files_endwith
                        file_path = os.path.join(dirpath, f)
                        
                        # db_names.append(name)
                        db_paths.append(file_path)

    return db_paths


In [26]:

images_extenstions = ['.png', '.jpg','.cr2', 'gif', 'bmp', '.tif', 'webp', '.heic', '.jpeg', '.JPEG', '.JPG' ]

img_paths = get_images_to_list(ROOT_DIRS, files_endwith=images_extenstions)

img_paths


['D:\\NEOM_PROJECT\\Marine_New_Data\\Islands flora and fauna baseline studies\\3. deliverables\\01 Reports\\June Survey\\N745_NEOM Islands_GIS Deliverables\\Shapefiles\\000_Premobilisation\\Basemaps\\AlFarsha.jpeg',
 'D:\\NEOM_PROJECT\\Marine_New_Data\\Islands flora and fauna baseline studies\\3. deliverables\\01 Reports\\June Survey\\N745_NEOM Islands_GIS Deliverables\\Shapefiles\\000_Premobilisation\\Basemaps\\Level1.jpeg',
 'D:\\NEOM_PROJECT\\Marine_New_Data\\Islands flora and fauna baseline studies\\3. deliverables\\01 Reports\\June Survey\\N745_NEOM Islands_GIS Deliverables\\Shapefiles\\000_Premobilisation\\Basemaps\\Level2.jpeg',
 'D:\\NEOM_PROJECT\\Marine_New_Data\\Islands flora and fauna baseline studies\\3. deliverables\\01 Reports\\June Survey\\N745_NEOM Islands_GIS Deliverables\\Shapefiles\\000_Premobilisation\\Basemaps\\MaktalAli.jpeg',
 'D:\\NEOM_PROJECT\\Marine_New_Data\\Islands flora and fauna baseline studies\\3. deliverables\\01 Reports\\June Survey\\N745_NEOM Islands_

In [27]:
len(img_paths)

5423

In [28]:
# open a sample image 
img = Image.open(
    r'D:\\NEOM_PROJECT\\Marine_New_Data\\Islands flora and fauna baseline studies\\3. deliverables\\01 Reports\\June Survey\\N745_NEOM Islands_GIS Deliverables\\Shapefiles\\000_Premobilisation\\Maps\\Sila Islands_Terrain.jpeg')

# img.show()

In [29]:
# function to extract meta data for images
def extract_image_metadata(
    image_paths,
    output_csv
):
    """
    Extracts metadata from image files using file system info,
    PIL image headers, EXIF (if present), and path-based inference.

    Parameters
    ----------
    image_paths : list[str]
        List of full paths to image files
    output_csv : str
        Path to save metadata CSV

    Returns
    -------
    pd.DataFrame
        Image metadata table
    """

    from PIL import Image, ExifTags
    import os
    import pandas as pd
    from datetime import datetime

    records = []

    # Reverse EXIF tag map once
    EXIF_TAGS = {v: k for k, v in ExifTags.TAGS.items()}

    for img_path in image_paths:

        file_name = os.path.basename(img_path)
        name_no_ext, ext = os.path.splitext(file_name)

        meta = {
            "image_path": img_path,
            "file_name": file_name,
            "file_extension": ext.lower(),
            "status": "success",
            "error": None
        }

        try:
            # ---- File system metadata ----
            stat = os.stat(img_path)

            meta["file_size_mb"] = round(stat.st_size / (1024 ** 2), 3)
            meta["created_time"] = datetime.fromtimestamp(stat.st_ctime)
            meta["modified_time"] = datetime.fromtimestamp(stat.st_mtime)

            # ---- Path-based metadata ----
            path_parts = img_path.split(os.sep)
            path_parts_lower = [p.lower() for p in path_parts]

            meta["Species"] = find_match(SPECIES_TYPES, path_parts_lower)
            meta["activity"] = find_match(ACTIVITY_TYPES, path_parts_lower)

            # ---- Filename-derived metadata ----
            tokens = name_no_ext.replace("-", "_").split("_")
            meta["filename_tokens"] = ", ".join(tokens)

            # ---- Image header metadata ----
            with Image.open(img_path) as img:
                meta["image_format"] = img.format
                meta["color_mode"] = img.mode
                meta["width_px"], meta["height_px"] = img.size
                meta["aspect_ratio"] = round(img.size[0] / img.size[1], 4)

                # ---- EXIF metadata (best effort) ----
                exif_data = img._getexif()
                if exif_data:
                    exif = {
                        ExifTags.TAGS.get(tag, tag): value
                        for tag, value in exif_data.items()
                        if tag in ExifTags.TAGS
                    }

                    meta["has_exif"] = True
                    meta["camera_make"] = exif.get("Make")
                    meta["camera_model"] = exif.get("Model")
                    meta["datetime_original"] = exif.get("DateTimeOriginal")
                    meta["gps_info"] = "GPSInfo" in exif
                else:
                    meta["has_exif"] = False
                    meta["camera_make"] = None
                    meta["camera_model"] = None
                    meta["datetime_original"] = None
                    meta["gps_info"] = False

        except Exception as e:
            meta["status"] = "failed"
            meta["error"] = str(e)

        records.append(meta)

    # ---- Create DataFrame ----
    meta_df = pd.DataFrame(records)

    # ---- Save CSV ----
    meta_df.to_csv(output_csv, index=False, encoding="utf-8")

    # return meta_df


In [30]:
extract_image_metadata(img_paths, OUTPUT_IMGS_METADATA_CSV)

## FOR SHAPE FILES

In [31]:
# check all shape files and geopackages
shape_exts = ['.shp', '.gpkg']

shp_paths = get_images_to_list(ROOT_DIRS, files_endwith=shape_exts)

shp_paths

['D:\\NEOM_PROJECT\\Marine_New_Data\\04_83504_KBD_Turtle_Tag_Combined\\KBD_Turtle_Tag_Core_Home_Range_Final_20240910.shp',
 'D:\\NEOM_PROJECT\\Marine_New_Data\\04_83504_KBD_Turtle_Tag_Combined\\KBD_Turtle_Tag_Data_Final_20240910.shp',
 'D:\\NEOM_PROJECT\\Marine_New_Data\\Cetacean Surveys\\305. Raw Data\\NEOM Cetacean Survey Results Shape FIles and Data\\._2022-01_All_Sightings.shp',
 'D:\\NEOM_PROJECT\\Marine_New_Data\\Cetacean Surveys\\305. Raw Data\\NEOM Cetacean Survey Results Shape FIles and Data\\._Acoustics.shp',
 'D:\\NEOM_PROJECT\\Marine_New_Data\\Cetacean Surveys\\305. Raw Data\\NEOM Cetacean Survey Results Shape FIles and Data\\._Coastal Effort Merge.shp',
 'D:\\NEOM_PROJECT\\Marine_New_Data\\Cetacean Surveys\\305. Raw Data\\NEOM Cetacean Survey Results Shape FIles and Data\\._Merge_Transect Effort.shp',
 'D:\\NEOM_PROJECT\\Marine_New_Data\\Cetacean Surveys\\305. Raw Data\\NEOM Cetacean Survey Results Shape FIles and Data\\._otherwildlife.shp',
 'D:\\NEOM_PROJECT\\Marine_New_

In [32]:
# how many shapefiles
len(shp_paths)

790

In [52]:
# shapefile metadata extraction function
def extract_shapefile_metadata(
    shp_paths,
    output_csv,
    crs=CRS
):
    """
    Reads shapefiles and extracts metadata safely.

    Parameters
    ----------
    shp_paths : list[str]
        List of full paths to shapefiles
    output_csv : str
        Path to save metadata CSV

    Returns
    -------
    pd.DataFrame
        Detailed metadata table
    """

    records = []

    for shp in shp_paths:

        layer_name = os.path.splitext(os.path.basename(shp))[0]

        meta = {
            "shapefile_path": shp,
            "layer_name": layer_name,
            "status": "success",
            "error": None
        }

        try:
            # ---- Read shapefile ----
            gdf = gpd.read_file(shp)

            if gdf.empty:
                raise ValueError("Shapefile contains no features")

            if gdf.crs is None:
                raise ValueError("Layer has no CRS defined")

            # ---- CRS handling ----
            epsg = gdf.crs.to_epsg()
            if epsg == 4326:
                gdf = gdf.to_crs(CRS)  # adjust if needed

            # ---- Geometry cleanup ----
            gdf["geometry"] = gdf.geometry.make_valid()

            geom = gdf.geometry.union_all()

            # ---- Oriented bounding box ----
            obb = geom.minimum_rotated_rectangle
            obb_coords = list(obb.exterior.coords)[:4]

            # ---- Spatial metadata ----
            meta["crs"] = str(gdf.crs)
            meta["epsg"] = gdf.crs.to_epsg()
            meta["geometry_types"] = ", ".join(sorted(gdf.geom_type.unique()))
            meta["bbox"] = list(gdf.total_bounds)
            meta["obb_bbox"] = obb_coords


            # ---- Feature-level metadata ----
            meta["feature_count"] = len(gdf)
            meta["has_geometry"] = "geometry" in gdf.columns

            # ---- Temporal metadata ----
            existing_date_cols = [col for col in DATE_COLUMNS if col in gdf.columns]
            meta["has_timestamp"] = len(existing_date_cols) > 0

            if meta["has_timestamp"]:
                for col in existing_date_cols:
                    gdf[col] = pd.to_datetime(gdf[col], errors="coerce")

                all_dates = pd.concat(
                    [gdf[col].dropna() for col in existing_date_cols],
                    ignore_index=True
                )

                if not all_dates.empty:
                    meta["min_date"] = all_dates.min()
                    meta["max_date"] = all_dates.max()
                else:
                    meta["min_date"] = None
                    meta["max_date"] = None
            else:
                meta["min_date"] = None
                meta["max_date"] = None

            # ---- Attribute metadata ----
            meta["field_count"] = len(gdf.columns)
            meta["field_names"] = ", ".join(gdf.columns)

            meta["field_types"] = ", ".join(
                f"{col}:{dtype}"
                for col, dtype in gdf.dtypes.items()
            )

            # ---- Path-based metadata ----
            shp_parts = shp.split(os.sep)
            shp_parts_lower = [p.lower() for p in shp_parts]

            meta["Species"] = find_match(SPECIES_TYPES, shp_parts_lower)
            meta["activity"] = find_match(ACTIVITY_TYPES, shp_parts_lower)

            # ---- Derived metadata ----
            meta["memory_mb"] = round(
                gdf.memory_usage(deep=True).sum() / (1024 ** 2), 3
            )

            # ---- Z / M detection ----
            try:
                meta["has_z"] = gdf.geometry.has_z.any()
            except Exception:
                meta["has_z"] = None

        except (DriverError, PermissionError) as e:
            meta["status"] = "skipped"
            meta["error"] = str(e)

        except Exception as e:
            meta["status"] = "failed"
            meta["error"] = str(e)

        records.append(meta)

    # ---- Create DataFrame ----
    meta_df = pd.DataFrame(records)

    # ---- Save CSV ----
    meta_df.to_csv(output_csv, index=False, encoding="utf-8")
    
    # return meta_df


In [ ]:
# get shp meta data to csv
extract_shapefile_metadata(shp_paths, output_csv=OUTPUT_SHP_METADATA_CSV)

## FOR CSV FILES


In [ ]:
# check all shape files
csv_exts = ['.csv', '.xlsx', '.xls']

csv_paths = get_images_to_list(ROOT_DIRS, files_endwith=csv_exts)

csv_paths

['D:\\NEOM_PROJECT\\Marine_New_Data\\Birds and Turtles Surveys AECOM\\305. Raw Data\\04_NEOM_FinalData_GDB_20211209\\Excel\\._Osprey_SootyFalcon_Observations.xls',
 'D:\\NEOM_PROJECT\\Marine_New_Data\\Birds and Turtles Surveys AECOM\\305. Raw Data\\04_NEOM_FinalData_GDB_20211209\\Excel\\._Other_Observations.xls',
 'D:\\NEOM_PROJECT\\Marine_New_Data\\Birds and Turtles Surveys AECOM\\305. Raw Data\\04_NEOM_FinalData_GDB_20211209\\Excel\\._Raptor_Satelite_Tagging.xls',
 'D:\\NEOM_PROJECT\\Marine_New_Data\\Birds and Turtles Surveys AECOM\\305. Raw Data\\04_NEOM_FinalData_GDB_20211209\\Excel\\._Sea_Shore_Bird_Observations.xls',
 'D:\\NEOM_PROJECT\\Marine_New_Data\\Birds and Turtles Surveys AECOM\\305. Raw Data\\04_NEOM_FinalData_GDB_20211209\\Excel\\._Tagged_Bird_Movements.xls',
 'D:\\NEOM_PROJECT\\Marine_New_Data\\Birds and Turtles Surveys AECOM\\305. Raw Data\\04_NEOM_FinalData_GDB_20211209\\Excel\\._Turtle_Observations.xls',
 'D:\\NEOM_PROJECT\\Marine_New_Data\\Birds and Turtles Surveys 

In [ ]:
# csv metadata extraction function
def extract_table_metadata(
    table_paths,
    output_csv
):
    """
    Extracts metadata from CSV and Excel files (.csv, .xlsx, .xls).

    Parameters
    ----------
    table_paths : list[str]
        List of full paths to CSV / Excel files
    output_csv : str
        Path to save metadata CSV

    Returns
    -------
    pd.DataFrame
        Tabular metadata table
    """

    import os
    import pandas as pd
    from datetime import datetime

    records = []

    for file_path in table_paths:

        file_name = os.path.basename(file_path)
        name_no_ext, ext = os.path.splitext(file_name)

        ext = ext.lower()

        meta = {
            "file_path": file_path,
            "file_name": file_name,
            "file_extension": ext,
            "status": "success",
            "error": None
        }

        try:
            # ---- File system metadata ----
            stat = os.stat(file_path)

            meta["file_size_mb"] = round(stat.st_size / (1024 ** 2), 3)
            meta["created_time"] = datetime.fromtimestamp(stat.st_ctime)
            meta["modified_time"] = datetime.fromtimestamp(stat.st_mtime)

            # ---- Path-based inference ----
            path_parts = file_path.split(os.sep)
            path_parts_lower = [p.lower() for p in path_parts]

            meta["Species"] = find_match(SPECIES_TYPES, path_parts_lower)
            meta["activity"] = find_match(ACTIVITY_TYPES, path_parts_lower)

            # ---- Filename parsing ----
            tokens = name_no_ext.replace("-", "_").split("_")
            meta["filename_tokens"] = ", ".join(tokens)

            # ---- Table-level metadata ----
            meta["row_count"] = None
            meta["column_count"] = None
            meta["column_names"] = None
            meta["column_types"] = None
            meta["sheet_count"] = None
            meta["sheet_names"] = None
            meta["has_timestamp"] = False
            meta["min_date"] = None
            meta["max_date"] = None

            # ---- CSV handling ----
            if ext == ".csv":
                df = pd.read_csv(file_path, nrows=1000)

                meta["row_count"] = sum(1 for _ in open(file_path, encoding="utf-8", errors="ignore")) - 1
                meta["column_count"] = len(df.columns)
                meta["column_names"] = ", ".join(df.columns)
                meta["column_types"] = ", ".join(
                    f"{c}:{t}" for c, t in df.dtypes.items()
                )

            # ---- Excel handling ----
            elif ext in [".xlsx", ".xls"]:
                xls = pd.ExcelFile(file_path)

                meta["sheet_count"] = len(xls.sheet_names)
                meta["sheet_names"] = ", ".join(xls.sheet_names)

                df = xls.parse(xls.sheet_names[0], nrows=1000)

                meta["column_count"] = len(df.columns)
                meta["column_names"] = ", ".join(df.columns)
                meta["column_types"] = ", ".join(
                    f"{c}:{t}" for c, t in df.dtypes.items()
                )

                meta["row_count"] = None  # avoid loading full sheets

            else:
                raise ValueError(f"Unsupported file type: {ext}")

            # ---- Temporal column detection ----
            date_cols = [c for c in DATE_COLUMNS if c in df.columns]

            if date_cols:
                meta["has_timestamp"] = True

                for c in date_cols:
                    df[c] = pd.to_datetime(df[c], errors="coerce")

                all_dates = pd.concat([df[c].dropna() for c in date_cols])

                if not all_dates.empty:
                    meta["min_date"] = all_dates.min()
                    meta["max_date"] = all_dates.max()

        except PermissionError as e:
            meta["status"] = "skipped"
            meta["error"] = str(e)

        except Exception as e:
            meta["status"] = "failed"
            meta["error"] = str(e)

        records.append(meta)

    # ---- Create DataFrame ----
    meta_df = pd.DataFrame(records)

    # ---- Save CSV ----
    meta_df.to_csv(output_csv, index=False, encoding="utf-8")

    # return meta_df


In [ ]:
# get all csv and excel tables meta data
extract_table_metadata(csv_paths, OUTPUT_CSV_METADATA_CSV)

c:\PERSONAL\UK PHD\NEOM_PROJECT\neom\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\PERSONAL\UK PHD\NEOM_PROJECT\neom\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\PERSONAL\UK PHD\NEOM_PROJECT\neom\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\PERSONAL\UK PHD\NEOM_PROJECT\neom\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\PERSONAL\UK PHD\NEOM_PROJECT\neom\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\PERSONAL\UK PHD\NEOM_PROJECT\neom\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Va